In [3]:
import sys
import os 
import pandas as pd
import numpy as np

import soepy

from dev_library import plot_basics_choices
from dev_library import plot_basics_wages
from dev_library import df_alignment

import pandas as pd
pd.set_option('display.max_rows', 500)
sys.path.insert(0, os.environ["PROJECT_DIR"] + "/estimations/peisenha_exploration")
os.chdir(os.environ["PROJECT_DIR"] + "/estimations/peisenha_exploration")


In [9]:
start = pd.read_pickle("start.soepy.pkl")
df_steps = pd.read_pickle("steps.respy.pkl")

In [76]:
df_step = df_steps.tail(31).loc[0, :]
df_step.drop(["fval", "fval"], inplace=True)
df_step.reset_index(inplace=True)
df_step.set_index(["category", "name"], inplace=True)
df_step.index.levels[0]

df_step.to_pickle("start.soepy.pkl")

Index(['benefits_base', 'const_wage_eq', 'delta', 'disutil_work', 'exp_accm',
       'exp_deprec', 'exp_returns', 'hetrg_unobs', 'mu', 'sd_wage_shock',
       'shares'],
      dtype='object', name='category')

In [61]:
df_step.index.levels[0]

Index(['benefits_base', 'const_wage_eq', 'delta', 'disutil_work', 'exp_accm',
       'exp_deprec', 'exp_returns', 'fval', 'hetrg_unobs', 'mu',
       'sd_wage_shock', 'shares'],
      dtype='object', name='category')

Index(['benefits_base', 'const_wage_eq', 'delta', 'disutil_work', 'exp_accm',
       'exp_deprec', 'exp_returns', 'fval', 'hetrg_unobs', 'mu',
       'sd_wage_shock', 'shares'],
      dtype='object', name='category')